In [1]:
from arctic import Arctic
from arctic.date import DateRange
from arctic.exceptions import NoDataFoundException
from arctic.date import DateRange
import pandas as pd
from ib_insync import IB, ContFuture, util, Future, Forex, Contract
from typing import Union
import pickle
from datetime import datetime, timedelta
import sys
sys.path.append('/home/tomek/ib_tools')
from datastore import ArcticStore

/home/tomek/.virtualenvs/ib/lib/python3.7/site-packages/arctic/store/_pandas_ndarray_store.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import DataFrame, Series, Panel


In [2]:
util.startLoop()
ib = IB()
ib.connect(port=4002, clientId=9)

<IB connected to 127.0.0.1:4002 clientId=9>

In [3]:
ib.isConnected()

True

In [ ]:
Arctic('localhost').list_libraries()

In [4]:
store = ArcticStore('TRADES_30_secs')

Library created, but couldn't enable sharding: no such command: 'enablesharding'. This is OK if you're not 'admin'


In [ ]:
keys = store.keys()
keys

In [ ]:
df = store.read('ESM0_CONTFUT', start_date='20180601', end_date='20200101')
df.loc['20180601 16:15:00':].head(10)

In [ ]:
df[df.index.duplicated()]

In [ ]:
resampled = df.resample('T').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum', 'barCount': 'sum'}).dropna()
resampled

In [ ]:
df.close.plot(figsize=(20,10), grid=True);

In [ ]:
from datastore import Store
old_store = Store()

In [ ]:
df1 = old_store.read('/cont/min/ES_20200320_GLOBEX_USD').sort_index(ascending=True).loc['20180601': '20191231']
df1

In [ ]:
df1[df1.index.duplicated()]

In [ ]:
df1.close.plot(figsize=(20,10), grid=True);

In [ ]:
comp = pd.DataFrame({'new': resampled.close, 'old': df1.close}).dropna()
comp['diff'] = comp['old'] - comp['new']
comp

In [ ]:
comp['diff'].plot(figsize=(20,10), grid=True);

In [ ]:
nq = ContFuture('NQ', exchange='globex')
ib.qualifyContracts(nq)

In [ ]:
from ib_insync import MarketOrder
order = MarketOrder('BUY', 1)

In [ ]:
ib.whatIfOrder(nq, order).commission

In [7]:
store.review('multiplier', 'commission')

,from,to,symbol,tradingClass,currency,min_tick,lastTradeDateOrContractMonth,name,multiplier,commission
6EM0_CONTFUT,2017-02-15 19:04:30,2020-06-10 15:13:00,EUR,6E,USD,5.000000e-05,20200615,European Monetary Union Euro,125000,2.470
6EU0_CONTFUT,2017-05-31 16:44:30,2020-06-25 11:07:00,EUR,6E,USD,5.000000e-05,20200914,European Monetary Union Euro,125000,2.470
6JM0_CONTFUT,2017-01-12 21:08:00,2020-06-10 15:13:00,JPY,6J,USD,5.000000e-07,20200615,JPY.USD Forex,12500000,2.470
6JU0_CONTFUT,2017-11-15 09:06:30,2020-06-25 11:07:00,JPY,6J,USD,5.000000e-07,20200914,JPY.USD Forex,12500000,2.470
BZZ0_CONTFUT,2016-02-02 15:09:30,2020-06-25 11:07:00,BZ,BZ,USD,1.000000e-02,20201030,Brent Crude Oil - Last Day,1000,1.220
CLN0_CONTFUT,2017-12-04 05:08:30,2020-06-15 12:37:00,CL,CL,USD,1.000000e-02,20200622,Light Sweet Crude Oil,1000,2.370
CLU0_CONTFUT,2019-02-15 00:00:00,2020-06-25 11:07:00,CL,CL,USD,1.000000e-02,20200820,Light Sweet Crude Oil,1000,2.370
ESM0_CONTFUT,2017-06-16 14:55:30,2020-06-10 15:13:00,ES,ES,USD,2.500000e-01,20200619,E-mini S&P 500,50,2.100
ESU0_CONTFUT,2017-08-17 14:16:30,2020-06-25 11:07:00,ES,ES,USD,2.500000e-01,20200918,E-mini S&P 500,50,2.100
FCEM0_CONTFUT,2018-01-02 09:08:30,2020-06-15 12:37:00,CAC40,FCE,EUR,5.000000e-01,20200619,CAC 40,10,1.200


In [ ]:
store.latest_contfutures(-2)

In [ ]:
store.read('YM_JUN_20_CONTFUT', start_date='20180601')

In [ ]:
ib.qualifyContracts(ContFuture('JPY', exchange='globex'))

In [ ]:
store.latest_contfutures(index = -2, field = 'symbol')

In [ ]:
volumes = {}
for symbol in ['EUR', 'JPY', 'GC', 'RTY', 'YM', 'NQ', 'ES']:
    df = store.contfuture(symbol, index=-2, field='symbol')
    volumes[symbol] = df.volume.rolling(90).sum().mean().round()
volumes

In [ ]:
store._contfutures_dict(field='symbol')

In [12]:
store.contfuture('EUR', field='symbol', index=-1, start_date='20180601').head(50)

,open,high,low,close,volume,average,barCount
date,,,,,,,
2018-06-01 00:00:00,1.17050,1.17050,1.17035,1.17035,63,1.170415,35
2018-06-01 00:00:30,1.17040,1.17040,1.17035,1.17040,16,1.170390,9
2018-06-01 00:01:00,1.17040,1.17050,1.17040,1.17050,18,1.170460,11
2018-06-01 00:01:30,1.17050,1.17055,1.17050,1.17055,4,1.170510,3
2018-06-01 00:02:00,1.17050,1.17055,1.17050,1.17055,12,1.170535,5
2018-06-01 00:02:30,1.17050,1.17050,1.17050,1.17050,11,1.170500,8
2018-06-01 00:03:00,1.17050,1.17055,1.17050,1.17055,4,1.170525,3
2018-06-01 00:03:30,1.17055,1.17055,1.17055,1.17055,6,1.170550,3
2018-06-01 00:04:00,1.17055,1.17060,1.17055,1.17060,4,1.170575,2


In [13]:
store.review('localSymbol')

,from,to,symbol,tradingClass,currency,min_tick,lastTradeDateOrContractMonth,name,localSymbol
6EM0_CONTFUT,2017-02-15 19:04:30,2020-06-10 15:13:00,EUR,6E,USD,5.000000e-05,20200615,European Monetary Union Euro,6EM0
6EU0_CONTFUT,2017-05-31 16:44:30,2020-06-25 11:07:00,EUR,6E,USD,5.000000e-05,20200914,European Monetary Union Euro,6EU0
6JM0_CONTFUT,2017-01-12 21:08:00,2020-06-10 15:13:00,JPY,6J,USD,5.000000e-07,20200615,JPY.USD Forex,6JM0
6JU0_CONTFUT,2017-11-15 09:06:30,2020-06-25 11:07:00,JPY,6J,USD,5.000000e-07,20200914,JPY.USD Forex,6JU0
BZZ0_CONTFUT,2016-02-02 15:09:30,2020-06-25 11:07:00,BZ,BZ,USD,1.000000e-02,20201030,Brent Crude Oil - Last Day,BZZ0
CLN0_CONTFUT,2017-12-04 05:08:30,2020-06-15 12:37:00,CL,CL,USD,1.000000e-02,20200622,Light Sweet Crude Oil,CLN0
CLU0_CONTFUT,2019-02-11 00:00:00,2020-06-25 11:07:00,CL,CL,USD,1.000000e-02,20200820,Light Sweet Crude Oil,CLU0
ESM0_CONTFUT,2017-06-16 14:55:30,2020-06-10 15:13:00,ES,ES,USD,2.500000e-01,20200619,E-mini S&P 500,ESM0
ESU0_CONTFUT,2017-08-17 14:16:30,2020-06-25 11:07:00,ES,ES,USD,2.500000e-01,20200918,E-mini S&P 500,ESU0
FCEM0_CONTFUT,2018-01-02 09:08:30,2020-06-15 12:37:00,CAC40,FCE,EUR,5.000000e-01,20200619,CAC 40,FCEM0


In [ ]:
store.contfuture('MES', field='symbol', index=-1, start_date='20190701')

In [5]:
from utilities import update_details

In [6]:
update_details(ib, store)

Started to throttle requests
Stopped to throttle requests


In [ ]:
store.read(nq, '20180701', end).head(10)

In [ ]:
date_range = DateRange('2018-05-19 00:00:00', '2019-12-31 00:00:00')

In [ ]:
start = '2018-05-19 00:00:00'
end = '2019-12-31 00:00:00'

In [ ]:
nq = ContFuture(conId=371749745, symbol='NQ', lastTradeDateOrContractMonth='20200918', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQU0', tradingClass='NQ')


In [ ]:
nq

In [ ]:
bars = ib.reqHistoricalData(
            nq,
            endDateTime= '20180602 00:00:00',
            durationStr='3 D',
            barSizeSetting='30 secs',
            whatToShow='TRADES',
            useRTH=False,
            formatDate=1)

In [ ]:
df = util.df(bars)
df

In [ ]:
store.read(nq, '20180701', end)

In [ ]:
l = store._contfutures_dict()
l

In [ ]:
x = {}
index = -2
for c, d in l.items():
    keys_list = list(d.keys())
    if len(keys_list) + index < 0:
        index = 0
    x[c] = d[keys_list[index]]
x

In [ ]:
list(d.keys())[0]

In [ ]:
store.latest_contfutures(-2)

In [8]:
store.contfuture_contract_object('EUR', -2)

TypeError: contfuture_contract_object() got an unexpected keyword argument 'start_date'

In [ ]:
df = store.read('ESM0_CONTFUT', '20180601', '20191231')
df

In [ ]:
(df['close'][-1] - df['open'][0])*20

In [ ]:
store.read_metadata('NQM0_CONTFUT')

In [ ]:
from collections import defaultdict
c = defaultdict(dict)
for k, v in d.items():
    for i in sorted(v):
        c[k].update({i: v[i]})  
c

In [ ]:
d

In [ ]:
c

In [ ]:
store.read(nq)

In [ ]:
%%timeit
store.date_range()

In [ ]:
store.read_object('NQU0_CONTFUT')

In [ ]:
store.read_metadata('NQU0_CONTFUT')

In [ ]:
store.read_object('YM_JUN_20_CONTFUT')

In [ ]:
store.latest_contfutures

In [ ]:
store.contfuture('NQ')

In [ ]:
def hist(days_at_a_time):
    dt = ''
    barsList = []
    i = 10/days_at_a_time
    #i=1
    while i>0:
        bars = ib.reqHistoricalData(
            nq,
            endDateTime=dt,
            durationStr=str(days_at_a_time) + ' D',
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=False,
            formatDate=1)
        #lib.append('edek', bars)
        if not bars:
            break
            dt -= timedelta(days=1)
            print(f'new dt: {dt}')
            continue
        barsList.append(bars)
        dt = bars[0].date
        print(dt)
        i -= 1
    allBars = [b for bars in reversed(barsList) for b in bars]
    df = util.df(allBars)
    return df

In [ ]:
bars = ib.reqHistoricalData(
            mes,
            endDateTime= '20190506 00:00:00',
            durationStr='1 D',
            barSizeSetting='30 secs',
            whatToShow='TRADES',
            useRTH=False,
            formatDate=1)

In [ ]:
nq

In [ ]:
nq.loc['20190101':]